In [1]:
!pip install neo4j

## Datasets

In [2]:
import pandas as pd

### `customers`

In [3]:
customers = pd.read_csv('datasets/customers.csv')

In [4]:
customers.count()

CIF              100
Age              100
EmailAddress     100
FirstName        100
LastName         100
PhoneNumber      100
Gender           100
Address          100
Country          100
JobTitle         100
CardNumber       100
AccountNumber    100
dtype: int64

In [5]:
customers.head()

,CIF,Age,EmailAddress,FirstName,LastName,PhoneNumber,Gender,Address,Country,JobTitle,CardNumber,AccountNumber
0,1,26,Greta_Swan6281@corti.com,Greta,Swan,6-254-223-4334,Female,"Armory Tunnel, 3699",Egypt,Bellman,348-14-6088,650-63-6154
1,2,41,Rebecca_Corbett5111@ovock.tech,Rebecca,Corbett,5-720-670-7047,Female,"Apostle Tunnel, 4850",Madagascar,Clerk,285-83-6186,875-13-4278
2,3,68,Dani_Flanders7065@qater.org,Dani,Flanders,8-150-728-6781,Female,"Bishop Street, 1298",The Gambia,Business Broker,811-84-3310,566-83-8044
3,4,68,Morgan_James9389@sveldo.biz,Morgan,James,7-215-157-6623,Female,"Wadham Avenue, 8018",Guinea-Bissau,Banker,773-80-8680,532-16-1750
4,5,51,Erick_Ingram7572@extex.org,Erick,Ingram,8-603-106-4884,Male,"Chamberlain Pass, 3481",Russia,Electrician,171-24-1530,683-00-5812


In [6]:
customers.dtypes

CIF               int64
Age               int64
EmailAddress     object
FirstName        object
LastName         object
PhoneNumber      object
Gender           object
Address          object
Country          object
JobTitle         object
CardNumber       object
AccountNumber    object
dtype: object

### `purchases`

In [7]:
purchases = pd.read_csv('datasets/purchases.csv')

In [8]:
purchases.count()

TransactionID       10000
CardNumber          10000
Merchant            10000
Amount              10000
PurchaseDatetime    10000
CardIssuer          10000
dtype: int64

In [9]:
purchases.head()

,TransactionID,CardNumber,Merchant,Amount,PurchaseDatetime,CardIssuer
0,751133,348-14-6088,Biolife Grup,9849.265683,2021-04-12 15:32:22Z,Bank of America
1,268445,285-83-6186,Facebook,2502.657530,2021-09-05 04:27:34Z,Visa
2,748511,811-84-3310,Comodo,6406.732352,2021-01-04 05:00:18Z,Chase
3,777162,773-80-8680,Demaco,12698.287510,2021-07-17 18:34:49Z,American Express
4,523244,171-24-1530,Erickson,13496.903670,2021-06-11 15:34:42Z,Chase


Find out if there is duplicate transaction id

In [10]:
purchases.groupby(['TransactionID'])['TransactionID'].size().sort_values(ascending=False)

TransactionID
739097    2
868861    2
639164    2
685230    2
466819    2
         ..
401599    1
401627    1
401764    1
401765    1
999987    1
Name: TransactionID, Length: 9958, dtype: int64

See the sample of duplicate tx ids

In [14]:
purchases[purchases['TransactionID'] == 739097]

,TransactionID,CardNumber,Merchant,Amount,PurchaseDatetime,CardIssuer,rank
6330,739097,358-87-6367,It Smart Group,13915.304730,2021-03-26 06:07:48Z,Capital One,1.0
8690,739097,804-18-1012,Apple Inc.,9277.671867,2021-06-19 07:06:41Z,Capital One,2.0


Drop duplicate transaction id with the lowest purchase amount, put it into a new dataframe called `purchases_cleaned`

In [12]:
purchases['rank'] = purchases.groupby(['TransactionID'])['Amount'].rank('dense', ascending=False)
purchases_cleaned = purchases[purchases['rank'] == 1]
purchases_cleaned = purchases_cleaned.drop(columns=['rank'])

In [13]:
purchases_cleaned.count()

TransactionID       9958
CardNumber          9958
Merchant            9958
Amount              9958
PurchaseDatetime    9958
CardIssuer          9958
dtype: int64

In [15]:
purchases_cleaned['PurchaseDatetime'] = pd.to_datetime(purchases_cleaned['PurchaseDatetime'], 
                                                      format='%Y-%m-%d %H:%M:%SZ')

In [16]:
purchases_cleaned.head()

,TransactionID,CardNumber,Merchant,Amount,PurchaseDatetime,CardIssuer
0,751133,348-14-6088,Biolife Grup,9849.265683,2021-04-12 15:32:22,Bank of America
1,268445,285-83-6186,Facebook,2502.657530,2021-09-05 04:27:34,Visa
2,748511,811-84-3310,Comodo,6406.732352,2021-01-04 05:00:18,Chase
3,777162,773-80-8680,Demaco,12698.287510,2021-07-17 18:34:49,American Express
4,523244,171-24-1530,Erickson,13496.903670,2021-06-11 15:34:42,Chase


In [17]:
purchases_cleaned.dtypes

TransactionID                int64
CardNumber                  object
Merchant                    object
Amount                     float64
PurchaseDatetime    datetime64[ns]
CardIssuer                  object
dtype: object

### `transfers`

In [19]:
transfers = pd.read_csv('datasets/transfers.csv')

In [20]:
transfers.count()

TransactionID            1000
SenderAccountNumber      1000
ReceiverAccountNumber    1000
Amount                   1000
TransferDatetime         1000
dtype: int64

In [21]:
transfers.head()

,TransactionID,SenderAccountNumber,ReceiverAccountNumber,Amount,TransferDatetime
0,287062,650-63-6154,430-04-5447,8584.094218,2021-02-21 16:21:49Z
1,377610,875-13-4278,650-63-6154,193772.025900,2021-04-21 03:09:06Z
2,513428,566-83-8044,875-13-4278,138147.425000,2021-05-21 22:03:26Z
3,571917,532-16-1750,566-83-8044,92918.537870,2021-07-22 14:35:48Z
4,418959,683-00-5812,532-16-1750,83355.477120,2021-02-03 07:53:22Z


Find out if there is duplicate transfer ids

In [22]:
transfers.groupby(['TransactionID'])['TransactionID'].size().sort_values(ascending=False)

TransactionID
835422    2
100710    1
688584    1
676257    1
678237    1
         ..
388598    1
388940    1
389629    1
391510    1
997278    1
Name: TransactionID, Length: 999, dtype: int64

In [23]:
transfers[transfers['TransactionID'] == 835422]

,TransactionID,SenderAccountNumber,ReceiverAccountNumber,Amount,TransferDatetime
247,835422,241-68-2480,560-48-1425,166905.3373,2021-07-20 07:31:25Z
277,835422,513-53-4378,256-10-2086,21815.7999,2021-05-22 14:40:47Z


Remove the duplicate transfer id with the lowest transaction amount and store it into a new dataframe `transfers_cleaned`

In [24]:
transfers_cleaned = transfers.drop(277)

transfers_cleaned[transfers_cleaned['TransactionID'] == 835422]

,TransactionID,SenderAccountNumber,ReceiverAccountNumber,Amount,TransferDatetime
247,835422,241-68-2480,560-48-1425,166905.3373,2021-07-20 07:31:25Z


In [25]:
transfers_cleaned.count()

TransactionID            999
SenderAccountNumber      999
ReceiverAccountNumber    999
Amount                   999
TransferDatetime         999
dtype: int64

In [26]:
transfers_cleaned['TransferDatetime'] = pd.to_datetime(transfers_cleaned['TransferDatetime'],
                                                      format='%Y-%m-%d %H:%M:%SZ')

In [27]:
transfers_cleaned.head()

,TransactionID,SenderAccountNumber,ReceiverAccountNumber,Amount,TransferDatetime
0,287062,650-63-6154,430-04-5447,8584.094218,2021-02-21 16:21:49
1,377610,875-13-4278,650-63-6154,193772.025900,2021-04-21 03:09:06
2,513428,566-83-8044,875-13-4278,138147.425000,2021-05-21 22:03:26
3,571917,532-16-1750,566-83-8044,92918.537870,2021-07-22 14:35:48
4,418959,683-00-5812,532-16-1750,83355.477120,2021-02-03 07:53:22


In [28]:
transfers_cleaned.dtypes

TransactionID                     int64
SenderAccountNumber              object
ReceiverAccountNumber            object
Amount                          float64
TransferDatetime         datetime64[ns]
dtype: object

## Data Model

![title](./images/Bank_transaction_purchase.png)

## Load data into Neo4j

Update the following variables based on your configuration.

In [35]:
uri='neo4j://localhost:7687'
username='neo4j'
password='pelesiran'
db='transaction2'

In [36]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri=uri, auth=(username,password))

In [32]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        try:
            self.__driver = GraphDatabase.driver(uri, auth=(user, password))
            print('Driver is successfully created.')
        except Exception as e:
            print(f'Failed to create the driver: {e}')
        
    def close(self):
        self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query,parameters=parameters))
        except Exception as e:
            print(f'Query failed: {e}')
        finally:
            if session is not None:
                session.close()
        return response
        

Create connection.

In [37]:
conn = Neo4jConnection(uri, username, password)

Driver is successfully created.


Create the database if not exist

In [38]:
conn.query(f'CREATE DATABASE {db} IF NOT EXISTS')

[]

Create constraints and indexes

In [39]:
conn.query('CREATE CONSTRAINT accounts IF NOT EXISTS FOR (a:Account) REQUIRE a.account_number IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT customers IF NOT EXISTS FOR (c:Customer) REQUIRE c.cif IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT accounts IF NOT EXISTS FOR (acc:Account) REQUIRE acc.cif IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT credit_cards IF NOT EXISTS FOR (cc:Credit_card) REQUIRE cc.card_number IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT merchants IF NOT EXISTS FOR (m:Merchant) REQUIRE m.name IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT countries IF NOT EXISTS FOR (co:Country) REQUIRE co.name IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT addresses IF NOT EXISTS FOR (a:Address) REQUIRE a.address_line IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT transfers IF NOT EXISTS FOR (t:Transfer) REQUIRE t.id IS UNIQUE',db=db)
conn.query('CREATE CONSTRAINT purchases IF NOT EXISTS FOR (p:Purchase) REQUIRE p.id IS UNIQUE',db=db)
conn.query('CREATE INDEX transfer_datetime IF NOT EXISTS FOR (t:Transfer) ON (t.datetime)',db=db)
conn.query('CREATE INDEX purchase_datetime IF NOT EXISTS FOR (p:Purchase) ON (p.datetime)',db=db)

[]

### Insert country nodes

In [40]:
query = '''
UNWIND $rows AS row
MERGE(co:Country {name:row})

RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':customers['Country'].unique()}, db=db)

[<Record total=78>]

### Insert `(Customer)-[:LIVES_AT]->(Address)-[:LOCATED_IN]->(Country)`

In [41]:
query = '''
UNWIND $rows AS row
MATCH(co:Country {name:row.Country})
MERGE (c:Customer {cif:row.CIF, age:row.Age, email_address:row.EmailAddress, first_name:row.FirstName, 
                  last_name:row.LastName, phone_number:row.PhoneNumber, gender:row.Gender, 
    job_title:row.JobTitle})
MERGE (a:Address {address_line:row.Address})
MERGE(c)-[:LIVES_AT]->(a)-[:LOCATED_IN]->(co)

RETURN count(*) AS total
'''

conn.query(query, parameters={'rows':customers.to_dict('records')},db=db)

[<Record total=100>]

### Insert merchant nodes

In [42]:
## Create merchant nodes

query = '''
UNWIND $rows AS row
MERGE(m:Merchant {name:row})

RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':purchases['Merchant'].unique()},db=db)

[<Record total=30>]

### Create `(Customer)-[:HAS]->(Credit_card)`

In [43]:
## Create (customer)-[:HAS]->(credit_card)

query = '''
UNWIND $rows AS row
MATCH(c:Customer {cif:row.CIF})
MERGE(c)-[:HAS]->(cc:Credit_card {card_number:row.CardNumber})

RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':customers.to_dict('records')}, db=db)

[<Record total=100>]

### Create `(Credit_card)-[:BUY]->(Purchase)<-[:SELL]-(merchant)`

In [44]:
## Create (credit_card)-[:PURCHASED_FROM]->(merchant)

query = '''
UNWIND $rows AS row
MATCH(cc:Credit_card {card_number:row.CardNumber}),(m:Merchant {name:row.Merchant})
MERGE(p:Purchase {id:row.TransactionID,amount:row.Amount,datetime:row.PurchaseDatetime})
MERGE(cc)-[:BUY]->(p)<-[:SELL]-(m) 
    
RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':purchases_cleaned.to_dict('records')},db=db)

[<Record total=9958>]

### add `issuer` property to `Credit_card` node

In [45]:
query = '''
UNWIND $rows AS row
MATCH(cc:Credit_card {card_number:row.CardNumber})
SET cc.issuer=row.CardIssuer

RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':purchases_cleaned.to_dict('records')},db=db)

[<Record total=9958>]

### Create `(customer)-[:HAS]->(account)`

In [46]:
## Create (customer)-[:HAS]->(account)

query = '''
UNWIND $rows AS row
MATCH(c:Customer {cif:row.CIF})
MERGE(c)-[:HAS]->(a:Account {account_number:row.AccountNumber})

RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':customers.to_dict('records')},db=db)

[<Record total=100>]

### Create `(account)-[:TRANSFER_IN]->(Transfer)<-[:TRANSFER_OUT]-(Account)`

In [47]:
query = '''
UNWIND $rows AS row
MATCH(a:Account {account_number:row.SenderAccountNumber}),(b:Account {account_number:row.ReceiverAccountNumber})
MERGE(t:Transfer {id:row.TransactionID,amount:row.Amount,datetime:row.TransferDatetime})
MERGE(a)-[:TRANSFER_IN]->(t)<-[:TRANSFER_OUT]-(b)

RETURN COUNT(*) AS total
'''

conn.query(query, parameters={'rows':transfers_cleaned.to_dict('records')},db=db)

[<Record total=999>]

Done